## Introduction
####  Data analysis & visualiztion on the skills of data scientists from the job description of 2 hiring websites

In [63]:
from __future__ import print_function
import pandas as pd
import numpy as np

# Text preprocessing
import os,re

# Disable warning of 3 types
import warnings

#Plotting
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import altair as alt
from altair import datum
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()


# Other utils
from tqdm import tqdm  # Progress bar
from datetime import datetime
from dateutil import parser

#EDA tools.
import dtale

# nlp text cleaning
import nltk
import re
import string
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer # or LancasterStemmer, RegexpStemmer, SnowballStemmer

# Transformers
from transformers import pipeline
import ipywidgets as widgets
from transformers import pipeline
from sentence_transformers import SentenceTransformer

# Clustering algorithms
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, MiniBatchKMeans
from sklearn.decomposition import LatentDirichletAllocation
from scipy.cluster.hierarchy import ward, dendrogram
from sklearn.metrics.pairwise import cosine_similarity

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, ENGLISH_STOP_WORDS
from nltk.stem.snowball import SnowballStemmer

# Visualizing text
import spacy
import scattertext

### Data clean & manipulation
- Remove the duplicated records.
- Remove the incorrect or inappropriate skills
- Combine the main and skills into one table: df_full

In [2]:
# load main table only including the jobs of data scientists
ds_file= './01_data/output/datascientists.csv'
df_main=pd.read_csv(ds_file)
df_main['id']=df_main['id'].astype('int64')

In [3]:
# Drop the duplicated job postings 
df_main.drop_duplicates(subset=['employer','description','title','location'],inplace=True)
# Drop the job posting with same id even the above would be a little different.
df_main.drop_duplicates(subset=['id'],inplace=True)

In [4]:
#removed space in country
df_main['country']=df_main['country'].apply(lambda x: x.strip())

In [5]:
# load skills produced by azure
az_skills= './01_data/output/az_skills.csv'
df_skills=pd.read_csv(az_skills)

In [6]:
df_skills=df_skills[df_skills['category'].isin(['Skill','Product','Person'])]

In [7]:
len(df_skills)

26602

In [8]:
# Drop the duplicated skills in the same job description
df_skills.drop_duplicates(subset=['id','skill'],inplace=True)

In [9]:
len(df_skills)

22743

In [10]:
# Decide to divide the job postings into 3 types:junior, senior, and others
def ds_level(title=''):
    jr = ["junior", "jr", "jr.","intern","internship","young","student","analyst","associate"]
    sr = ["sr.","sr","senior","lead","leading","principal","president"]

    if any(x in title.lower() for x in jr):
        return "junior"

    if any(x in title.lower() for x in sr):
        return "senior"
    
    return "others"


In [11]:
df_main['type']=df_main['title'].apply(lambda x: ds_level(x))

In [12]:
# Change string to datetime
df_main['posting_date']=df_main['posting_date'].apply(lambda x: parser.parse(x))

In [13]:
# Perform EDA to check main table
#d = dtale.show(df_full)
#d.open_browser()

#### Refine the skills extracted from Azure

In [14]:
# The list of person to be excluded.
xl_prd=[
        'food coupons', 'coffee', 'room', 'snacks',
      'computer screens', 'computers', 'printers',
       'copiers', 'computer', 'pool table', 'beers',
       'soft drinks', 'wine', 'mac', 'pc', 'nintendo switch',
       'big screen tv', 'mario kart', 'holidays', 'car', 'cycle',
       'machine',
       'office suite',
       'ph', 'd', 'drinks', 'watson', 'breakfast', 'fruits', 'advanced',
       'artefact',  'suite',
       'tv', 'headphones', 'notebooks', 'medicines',
       'beverages', 'vehicle',
       'infrastructure', 'food',  'celery',
       'ansible', 'espresso', 
       'petrel', 'macbook', 'books', 'vinted', 'bikes', 'home',
       'travel insurance', 'earthsofts equis', 'power', 'journals',
       'big machines', 
        'meal',  'meal vouchers',
       'laptop', 
       'textbooks', 'nail polish', 'fridge',  'company car',
       'book', 'goodies', 'fresh fruits', 
        'refreshments', 'plan',
       'disability insurance', 'free drinks', 'fruit', 
      'notebook',
       'passport',
       'opportunity',
       'artificial barriers',
       'tessian anc headphones', 'coffee machine', 'hummus',
       'dinner', 
       'apparel', 'uniform', 'restaurant tickets', 'transport tickets',
       'gear',   'pizza',
       'beer', 'vouchers', 
        'dress',
        'conference tickets', 'cell phone', 'sport vouchers',
       'consumer', 'phone',
        'friday drinks',
       'beanbags', 'guitars', 'table football',
       'meals', 
       'shiny',
       'satellite', 'desks', 
       'mobile phone', 
       'outlook', 'word', 
       'jupiter', 
       'barista', 'coffee machines', 
       'furniture',  'health care plan',
       'death service plan', 'cycle to work scheme', 
       'cinema tickets', 
     'company', 
       'kitchen', 'gourmet coffee', 'teas', 'leaseauto', 'healthy snacks',
       'pc computing',  'hd tv', 'pass', 
         'bicycle', 'plans',
       'oil', 'gas', 
        'fast food', 'ship',
       'gas turbines', 'cntk', 'nvidia tx2,', 'nvidia xavier',
       'sap', 'db2.']

In [15]:
# The list of person to be excluded.
xl_per=['ltat de lart', 'john doe',
        'monte', 'tiki', 'san francisco', 'roche', 'jhu',
      'veronika grollova', 'frederik norgaard', 'kennis van',
       'balderton']

In [16]:
df_skills['skill']=df_skills['skill'].apply(lambda x: None if x in (xl_prd+xl_per) else x)

In [17]:
len(df_skills)

22743

In [18]:
df_skills.dropna(subset=['skill'],inplace=True)

In [19]:
len(df_skills)

22376

In [20]:
# Export skills for manual refining
df_skill_cnt=df_skills.groupby(['skill']).count()['id'].reset_index().sort_values(['id'],ascending=False)
df_skill_cnt.to_csv('skills.csv')

In [21]:
# load skills refined by manual, the csv's columns are changed to: id, skill,count, keep 
refined_file= './01_data/manual/skills_refined.csv'
df_skills_r=pd.read_csv(refined_file)

In [22]:
len(df_skills_r)

4547

In [23]:
df_skills

,Unnamed: 0,id,skill,category,confidence score
0,0,4.148184e+09,technologie-themen,Skill,0.84
1,1,4.148184e+09,technologie-themen,Skill,0.84
2,2,4.203393e+09,artificial intelligence,Skill,0.91
3,3,4.203393e+09,machine learning,Skill,0.83
7,7,4.203393e+09,datasets,Skill,0.80
...,...,...,...,...,...
33281,33281,4.147913e+09,decision tree,Skill,0.95
33282,33282,4.147913e+09,random,Skill,0.58
33283,33283,4.147913e+09,neural network,Skill,0.99
33284,33284,4.189587e+09,machine learning,Skill,1.00


In [24]:
df_skills=df_skills.merge(df_skills_r,how='left',right_on=['skill'],left_on=['skill'])

In [25]:
df_skills.dropna(subset=['keep'],inplace=True)

In [26]:
df_skills

,Unnamed: 0,id,skill,category,confidence score,index,count,keep
2,2,4.203393e+09,artificial intelligence,Skill,0.91,298.0,46.0,y
3,3,4.203393e+09,machine learning,Skill,0.83,2414.0,479.0,y
4,7,4.203393e+09,datasets,Skill,0.80,1346.0,73.0,y
5,8,4.203393e+09,cybersecurity,Skill,0.87,1075.0,2.0,y
6,9,4.203393e+09,developing,Skill,0.62,1459.0,96.0,y
...,...,...,...,...,...,...,...,...
22370,33280,4.147913e+09,machine learning,Skill,0.98,2414.0,479.0,y
22371,33281,4.147913e+09,decision tree,Skill,0.95,1369.0,5.0,y
22373,33283,4.147913e+09,neural network,Skill,0.99,2779.0,7.0,y
22374,33284,4.189587e+09,machine learning,Skill,1.00,2414.0,479.0,y


In [27]:
# Produce the refined main table
df_full=df_skills.merge(df_main,how='left',left_on=['id'],right_on=['id'])[['id', 'type','posting_date', 'description', 'title', 'country',
       'employer', 'industry', 'source', 'skill']]

In [28]:
df_full

,id,type,posting_date,description,title,country,employer,industry,source,skill
0,4.203393e+09,others,2019-09-27,*artificial intelligence/ machine learning dat...,AI/ML Data Scientist,United Kingdom,Tec Partners,unclassified,Glassdoor,artificial intelligence
1,4.203393e+09,others,2019-09-27,*artificial intelligence/ machine learning dat...,AI/ML Data Scientist,United Kingdom,Tec Partners,unclassified,Glassdoor,machine learning
2,4.203393e+09,others,2019-09-27,*artificial intelligence/ machine learning dat...,AI/ML Data Scientist,United Kingdom,Tec Partners,unclassified,Glassdoor,datasets
3,4.203393e+09,others,2019-09-27,*artificial intelligence/ machine learning dat...,AI/ML Data Scientist,United Kingdom,Tec Partners,unclassified,Glassdoor,cybersecurity
4,4.203393e+09,others,2019-09-27,*artificial intelligence/ machine learning dat...,AI/ML Data Scientist,United Kingdom,Tec Partners,unclassified,Glassdoor,developing
...,...,...,...,...,...,...,...,...,...,...
17194,4.147913e+09,others,2019-08-21,algorithmes de data mining.algorithmes de mach...,D/ASE/DATASCIENTIST - Ingénieur Data Scientist...,France,Groupe SII,IT Services,Glassdoor,machine learning
17195,4.147913e+09,others,2019-08-21,algorithmes de data mining.algorithmes de mach...,D/ASE/DATASCIENTIST - Ingénieur Data Scientist...,France,Groupe SII,IT Services,Glassdoor,decision tree
17196,4.147913e+09,others,2019-08-21,algorithmes de data mining.algorithmes de mach...,D/ASE/DATASCIENTIST - Ingénieur Data Scientist...,France,Groupe SII,IT Services,Glassdoor,neural network
17197,4.189587e+09,others,2019-08-21,machine learning et deep learning.librairies d...,Stage : Assistant Data Scientist - Moteur de r...,France,Crédit Agricole,Investment Banking & Asset Management,Glassdoor,machine learning


In [29]:
#Remove records which are duplicated and removed in main, but still exists in skill table
df_full.dropna(subset=['description'],inplace=True)

In [53]:
# data clean for data jobs
data_file= './01_Data/Output/datajobs.csv'
df_data=pd.read_csv(data_file)

In [55]:
# Drop the duplicated job postings 
df_data.drop_duplicates(subset=['employer','description','title','location'],inplace=True)
# Drop the job posting with same id even the above would be a little different.
df_data.drop_duplicates(subset=['id'],inplace=True)

In [56]:
# drop na
df_data.dropna(subset=['description'],inplace=True)
# Change string to datetime
df_data['posting_date']=df_data['posting_date'].apply(lambda x: parser.parse(x))

### Data analysis
- Jobs based analysis: by countries, by type(level), by industries
- Skills based analysis: by countries, by level, by industries.
- Seek to combine the above.
- Keywords anaysis in scattertext between Sr. and Jr.

##### Job distribution

In [50]:
df_full.groupby(['country']).count()['id'].sort_values(ascending=False)[0:10]

country
United States     2301
India             1941
United Kingdom    1468
Canada            1460
Germany           1271
Singapore         1084
France             484
Israel             404
Switzerland        400
China              381
Name: id, dtype: int64

In [49]:
df_full.groupby(['industry']).count()['id'].sort_values(ascending=False)[0:10]

industry
unclassified                               2551
Internet                                   1539
Computer Hardware & Software               1471
IT Services                                1439
Consulting                                 1191
Biotech & Pharmaceuticals                  1068
Enterprise Software & Network Solutions     989
Accounting                                  579
Investment Banking & Asset Management       525
Staffing & Outsourcing                      367
Name: id, dtype: int64

In [32]:
df_full.groupby(['type']).count()['id'].sort_values(ascending=False)

type
senior    11907
others     3137
junior     1959
Name: id, dtype: int64

In [60]:
df_main.groupby(df_main['posting_date'].dt.to_period("M")).count()['id'].sort_values(ascending=False)

posting_date
2019-10    475
2019-11    274
2019-09      7
2019-08      5
2019-07      3
2019-06      1
Freq: M, Name: id, dtype: int64

In [61]:
df_data.groupby(df_data['posting_date'].dt.to_period("M")).count()['id'].sort_values(ascending=False)

posting_date
2019-10    1980
2019-11    1243
2019-09      28
2019-08      22
2019-07       8
2019-06       4
2018-02       2
2019-04       1
Freq: M, Name: id, dtype: int64

##### Skills distribution

In [48]:
df_full.groupby(['country']).count()['skill'].sort_values(ascending=False)[0:10]

country
United States     2301
India             1941
United Kingdom    1468
Canada            1460
Germany           1271
Singapore         1084
France             484
Israel             404
Switzerland        400
China              381
Name: skill, dtype: int64

In [47]:
df_full.groupby(['industry']).count()['skill'].sort_values(ascending=False)[0:10]

industry
unclassified                               2551
Internet                                   1539
Computer Hardware & Software               1471
IT Services                                1439
Consulting                                 1191
Biotech & Pharmaceuticals                  1068
Enterprise Software & Network Solutions     989
Accounting                                  579
Investment Banking & Asset Management       525
Staffing & Outsourcing                      367
Name: skill, dtype: int64

In [36]:
df_full.groupby(['type']).count()['skill'].sort_values(ascending=False)

type
senior    11907
others     3137
junior     1959
Name: skill, dtype: int64

In [37]:
len(df_main['id'].unique())

765

In [38]:
len(set(df_main['id'].unique())-set(df_skills['id'].unique()))

35

In [39]:
not_list=list(set(df_main['id'].unique())-set(df_skills['id'].unique()))

In [40]:
len(df_skills['id'].unique())

741

In [41]:
# The reason that no. of id in df_full is less than df_main is:
# 1) No skills have been extracted by azure.
# 2) Extracted skills have been removed manually due to inappropriateness
# The detail is:
# df_main[df_main['id'].isin(not_list)]

In [42]:
df_full.groupby(['id']).count()['skill'].describe()

count    730.000000
mean      23.291781
std       14.184985
min        1.000000
25%       13.000000
50%       22.000000
75%       31.000000
max       80.000000
Name: skill, dtype: float64

In [43]:
df_full.groupby(['type']).count()['skill']

type
junior     1959
others     3137
senior    11907
Name: skill, dtype: int64

In [44]:
df_full.groupby(['type','id']).count()['skill'].describe()

count    730.000000
mean      23.291781
std       14.184985
min        1.000000
25%       13.000000
50%       22.000000
75%       31.000000
max       80.000000
Name: skill, dtype: float64

In [46]:
df_full.groupby(['skill']).count()['id'].sort_values(ascending=False)[0:10]

skill
machine learning    474
statistics          361
computer science    348
python              347
algorithms          227
mathematics         223
engineering         219
communication       199
analytics           191
r                   179
Name: id, dtype: int64

In [72]:
df_type

,type,skill,cnt
0,senior,machine learning,342
1,senior,statistics,264
2,senior,computer science,245
3,senior,python,236
4,senior,algorithms,171
...,...,...,...
4512,others,sec,1
4513,others,segmentation analysis,1
4514,others,segregation,1
4515,others,seismic interpretation,1


In [97]:
top_n=10
df_type=df_full.groupby(['type','skill']).count()['id'].sort_values(ascending=False)
df_type=df_type.reset_index()
df_type.columns=['type','skill','cnt']
df_sr=df_type[df_type['type']=='senior'].sort_values(['cnt'],ascending=False)[0:top_n]
df_jr=df_type[df_type['type']=='junior'].sort_values(['cnt'],ascending=False)[0:top_n]
df_ot=df_type[df_type['type']=='others'].sort_values(['cnt'],ascending=False)[0:top_n]

In [134]:
    height=200
    width=200
    s_chart=alt.Chart(df_sr
                         ).mark_bar().encode(
        y=alt.Y('skill:N', sort='-x',title='skill'),
        x=alt.X('cnt:Q',title=' # of count',axis=alt.Axis( 
                                   labelAngle=-45, 
                                   labelOverlap=False)),
        tooltip=[
                 alt.Tooltip('cnt:Q', format='d'),
                ]
    ).properties(
        height=height, width=width,
    )

    j_chart=alt.Chart(df_jr
                         ).mark_bar().encode(
        y=alt.Y('skill:N', sort='-x',title='skill'),
        x=alt.X('cnt:Q',title=' # of count',axis=alt.Axis( 
                                   labelAngle=-45, 
                                   labelOverlap=False)),
        tooltip=[
                 alt.Tooltip('cnt:Q', format='d'),
                ]
    ).properties(
        height=height, width=width,
    )
    o_chart=alt.Chart(df_ot
                         ).mark_bar().encode(
        y=alt.Y('skill:N', sort='-x',title='skill'),
        x=alt.X('cnt:Q',title=' # of count',axis=alt.Axis( 
                                   labelAngle=-45, 
                                   labelOverlap=False)),
        tooltip=[
                 alt.Tooltip('cnt:Q', format='d'),
                ]
    ).properties(
        height=height, width=width,
    )
 
    (s_chart | j_chart | o_chart).properties(
        title='Top skills by levels').configure_view(
        strokeOpacity=0 # Remove the border of chart title
    ).configure_concat(
        spacing=30 # Padding among all charts vertically
    ).configure_axisY(  # Change y title to horizontally
        titleAngle=0,
        titleAlign="right",
        titleY=-10,
        titleX=-20,
    )


alt.HConcatChart(...)

In [142]:
df_type.groupby('type').head(10)

,type,skill,cnt
0,senior,machine learning,342
1,senior,statistics,264
2,senior,computer science,245
3,senior,python,236
4,senior,algorithms,171
5,senior,mathematics,156
6,senior,engineering,155
7,senior,analytics,141
8,senior,communication,136
9,senior,r,124


In [176]:
top_n=20
b_height=300
b_width=700
title_str=" Top %d Skills By levels" %(top_n)
alt.Chart(df_type.groupby('type').head(top_n)).mark_bar().encode(
    y=alt.X('cnt:Q', title='# of count'),
    x=alt.Y('skill:O',sort='-y',title='skill',axis=alt.Axis( 
                                   labelAngle=-40, 
                                   labelOverlap=False)),
    color=alt.Color('type', scale=alt.Scale(scheme='set2')),
    order=alt.Order(
      # Sort the segments of the bars by this field
      'type',
      sort='ascending'
    )
).properties(height=b_height, width=b_width,
        title = alt.TitleParams(text = title_str,
                                            align='center',
                                            font = 'Ubuntu Mono', 
                                            fontSize = 20, 
                                            color = '#3E454F'
                                            )
    )

alt.Chart(...)

In [90]:
# Using scattertext to visualize the skills by types
nlp = spacy.load('en_core_web_sm')

In [91]:
    corpus = (scattertext.CorpusFromPandas(df_full,
                                           category_col='type', 
                                           text_col='skill',
                                           nlp=nlp)
              .build()
              .remove_terms(nlp.Defaults.stop_words, ignore_absences=True)
              )

In [92]:
html = scattertext.produce_scattertext_explorer(
                   corpus,
                   category='senior',
                   category_name='senior',
                   not_category_name=['junior'],
                   width_in_pixels=1000,
                   )

In [94]:
#open("ds_skills.html", 'wb').write(html.encode('utf-8'))
#with open("ds_skills.html", 'w') as outf: outf.write(html)
from IPython.display import IFrame    
display(IFrame("ds_skills.html", width=900, height=850))

In [95]:
df = corpus.get_term_freq_df()

In [96]:
df

,others freq,senior freq,junior freq
term,,,
artificial,13,35,7
intelligence,17,53,12
artificial intelligence,10,29,7
machine,98,396,54
learning,152,573,77
...,...,...,...
efficiency measurement,0,0,1
computing systems,0,2,0
technique,0,2,0
